<a href="https://colab.research.google.com/github/PBICF/PH7/blob/master/LLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft torch pandas -q

Load Model

In [ ]:
from transformers import pipeline

model_name = "Qwen/Qwen2.5-3B-Instruct"

# ask_llm = pipeline(
#     model= model_name,
#     device="cuda"
# )

# print(ask_llm("What is the meaning of LAP?")[0]["generated_text"])

Load Raw Dataset

In [ ]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd

vol1 = load_dataset(
    "csv",
    data_files="/content/drive/MyDrive/datasets/dataset-vol1.csv",
    split="train"
)

vol2 = load_dataset(
    "csv",
    data_files="/content/drive/MyDrive/datasets/dataset-vol2.csv",
    split="train"
)

merged_dataset = concatenate_datasets([vol1, vol2])
# merged_dataset.select(range(10)).to_pandas()[["rule_numbers", "question", "answer"]]

ml_dataset = merged_dataset.map(
    lambda x: {
        "instruction": "Answer the question using the given context.",
        "input": f"Rule {x['rule_numbers']}:\n{x['context']}\n\nQuestion:\n{x['question']}",
        "output": x["answer"]
    },
    remove_columns=merged_dataset.column_names
)

# ml_dataset.select(range(5)).to_pandas()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/797 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
def preprocess(sample):
    sample = (
        f"### Instruction:\n{sample['instruction']}\n\n"
        f"### Input:\n{sample['input']}\n\n"
        f"### Response:\n {sample["output"]}"
    )

    tokenized = tokenizer(
        sample,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [ ]:
data = ml_dataset.map(preprocess)

Map:   0%|          | 0/797 [00:00<?, ? examples/s]

LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Training / Fine Tuning

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


Train the Model

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.75 GiB is free. Process 3599 has 9.99 GiB memory in use. Of the allocated memory 9.78 GiB is allocated by PyTorch, and 90.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()